In [1]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import glob
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [2]:
FOLDER_MINUTES_TXT = "./data/processed/copom_minutes_txt"
FOLDER_MINUTES_LEMMATIZED = "./data/processed/copom_minutes_lemmatized"

In [3]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN

In [4]:
def preprocess_text(text):

    text = text.lower()

    tokens = word_tokenize(text)
    stop_words_english = set(stopwords.words('english'))
    pos_tagged_tokens = nltk.pos_tag(tokens)

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = []

    for word, tag in pos_tagged_tokens:
        if word not in stop_words_english and word.isalpha() and len(word) > 1:
            wordnet_pos = get_wordnet_pos(tag)
            lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
            lemmatized_tokens.append(lemma)
            
    return lemmatized_tokens

In [5]:
filepaths = glob.glob(f"{FOLDER_MINUTES_TXT}/*.txt")

for path in filepaths:
    filename = os.path.basename(path)
        
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
        
    lemm_texts = [preprocess_text(p) for p in paragraphs]

    with open(f"{FOLDER_MINUTES_LEMMATIZED}/{filename}", 'w', encoding='utf-8') as f:
        for paragraph in lemm_texts:
            line = ' '.join(paragraph)
            f.write(line + '\n')